# Flop C-Bet Explorer

Use this notebook to explore how different flop continuation-bet sizes correlate with the bettor's made hand or draw strength. Edit the configuration cell to experiment with custom sizing buckets or category groupings.

Workflow:
1. Adjust `BUCKETS`, `PRIMARY_GROUPS`, or `DRAW_FLAGS` below.
2. Rerun the helper cell and the summary cell to see the updated breakdown.
3. Optional: enable caching or convert events to a pandas DataFrame for ad-hoc filtering.

> Buckets use left-inclusive, right-exclusive ranges (e.g. a 0.25 bet falls into [0.25, 0.40)).


In [1]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    Path(r"T:\Dev\ignition\drivehud\drivehud.db"),
    Path("/mnt/t/Dev/ignition/drivehud/drivehud.db"),
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "cbet_events.json"

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")


In [2]:
import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from analysis.cbet_utils import (
    BASE_PRIMARY_CATEGORIES,
    DEFAULT_DRAW_FLAGS,
    available_primary_categories,
    events_to_dataframe,
    load_cbet_events,
    summarize_events,
    response_events,
)
from analysis.turn_utils import (
    TURN_BUCKETS as TURN_BUCKETS_TURN,
    load_turn_events as load_turn_events_turn,
    turn_response_events as turn_response_events_turn,
)


In [3]:
# --- Configuration ---
RAW_BUCKET_BOUNDS = [
    (0.00, 0.25),
    (0.25, 0.40),
    (0.40, 0.60),
    (0.60, 0.80),
    (0.80, 1.00),
    (1.00, 1.25),
    (1.25, float("inf")),
]

def _bucket_label(low: float, high: float) -> str:
    if high == float("inf"):
        return f">={low:.2f}"
    return f"[{low:.2f}, {high:.2f})"

BUCKETS = [(low, high, _bucket_label(low, high)) for (low, high) in RAW_BUCKET_BOUNDS]
del _bucket_label

PRIMARY_GROUPS = {cat: [cat] for cat in BASE_PRIMARY_CATEGORIES}
DRAW_FLAGS = DEFAULT_DRAW_FLAGS.copy()

# Set to True to rebuild the cache after changing parsing logic.
FORCE_RELOAD = True

TURN_CACHE_PATH = PROJECT_ROOT / "analysis/cache/turn_events.json"
TURN_FORCE_RELOAD = False



In [4]:
events = load_cbet_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
print(f"Loaded {len(events)} c-bet events.")
available_categories = available_primary_categories(events)
print("Observed primary categories:", ", ".join(available_categories))

try:
    turn_events = load_turn_events_turn(DB_PATH, cache_path=TURN_CACHE_PATH, force=TURN_FORCE_RELOAD)
    turn_responses = turn_response_events_turn(turn_events)
except FileNotFoundError:
    turn_events = []
    turn_responses = []



Loaded 5847 c-bet events.
Observed primary categories: Air, Bottom Pair, Flush, Full House, Middle Pair, Overpair, Quads, Straight, Top Pair, Trips/Set, Two Pair, Underpair


In [5]:
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

DEFAULT_DROP_COLUMNS = {"Range", "Made Flush", "Made Straight"}

GROUPED_PRIMARY_ORDER = [
    ("Events", None),
    ("Air", ["Air"]),
    ("Weak Pair", ["Underpair", "Bottom Pair", "Middle Pair"]),
    ("Top Pair", ["Top Pair"]),
    ("Overpair", ["Overpair"]),
    ("Two Pair", ["Two Pair"]),
    ("Trips/Set", ["Trips/Set"]),
    ("Monster", ["Straight", "Flush", "Full House", "Quads"]),
    ("Draw", ["Flush Draw", "OESD/DG"]),
]

RESPONSE_GROUP_ORDER = [
    ("Events", None),
    ("Fold", ["Fold"]),
    ("Call", ["Call"]),
    ("Raise", ["Raise"]),
    ("Continue", ["Call", "Raise"]),
]

def _gradient_cmap(base_color: str) -> LinearSegmentedColormap:
    name = f'gradient_{base_color.strip("#")}'
    return LinearSegmentedColormap.from_list(name, ['#ffffff', base_color])

def _summary_records_to_df(records, drop_columns=None):
    if not records:
        return pd.DataFrame()
    df = pd.DataFrame(records)
    drop_columns = drop_columns or set()
    for col in drop_columns:
        if col in df.columns:
            df = df.drop(columns=[col])
    df = df.set_index("Bucket").T
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

def _style_heatmap(df, title, base_color="#1f77b4", fmt="{:.1f}", special_columns=None):
    if df.empty:
        return None
    data_index = df.index.difference(["Events"])
    styled = df.style.format(fmt)
    if not data_index.empty:
        styled = styled.background_gradient(
            cmap=_gradient_cmap(base_color),
            axis=None,
            subset=pd.IndexSlice[data_index, :]
        )
    if special_columns:
        first = special_columns[0]
        if first in df.columns:
            col_idx = df.columns.get_loc(first)
            styles = [
                {
                    'selector': f'th.col_heading.level0.col{col_idx}',
                    'props': [('border-left', '2px solid #64748b')]
                },
                {
                    'selector': f'td.col{col_idx}',
                    'props': [('border-left', '2px solid #64748b')]
                },
            ]
            styled = styled.set_table_styles(styles, overwrite=False)
    return styled.set_caption(title)

def _aggregate_grouped(df, group_order):
    rows = []
    for label, members in group_order:
        if members is None:
            if "Events" in df.index:
                rows.append((label, df.loc["Events"]))
            continue
        available = [member for member in members if member in df.index]
        if not available:
            continue
        rows.append((label, df.loc[available].sum()))
    if not rows:
        return pd.DataFrame()
    data = pd.DataFrame([series for _, series in rows], index=[label for label, _ in rows])
    return data

def display_heatmap_tables(summary_records, title, *, base_color="#1f77b4", group_color="#2ca25f", special_rows=None):
    records = list(summary_records)
    special_columns = []
    if special_rows:
        records.extend(special_rows)
        special_columns = [row['Bucket'] for row in special_rows if row.get('Bucket')]
    base_df = _summary_records_to_df(records, DEFAULT_DROP_COLUMNS)
    styled = _style_heatmap(base_df, title, base_color=base_color, special_columns=special_columns)
    if styled is not None:
        display(styled)
    grouped_df = _aggregate_grouped(base_df, GROUPED_PRIMARY_ORDER)
    styled_grouped = _style_heatmap(grouped_df, f"{title} (Grouped)", base_color=group_color, special_columns=special_columns)
    if styled_grouped is not None:
        display(styled_grouped)

def summarize_responses_df(responses_df, buckets):
    summary = []
    for low, high, label in buckets:
        subset = responses_df[(responses_df['ratio'] >= low) & (responses_df['ratio'] < high)]
        if subset.empty:
            summary.append({
                'Bucket': label,
                'Events': 0,
                'Fold': 0.0,
                'Call': 0.0,
                'Raise': 0.0,
                'Continue': 0.0,
            })
            continue
        events_count = len(subset)
        response_counts = subset['response'].value_counts()
        continue_count = int(response_counts.get('Call', 0) + response_counts.get('Raise', 0))
        summary.append({
            'Bucket': label,
            'Events': events_count,
            'Fold': response_counts.get('Fold', 0) / events_count * 100,
            'Call': response_counts.get('Call', 0) / events_count * 100,
            'Raise': response_counts.get('Raise', 0) / events_count * 100,
            'Continue': continue_count / events_count * 100,
        })
    return summary


In [6]:
if 'df' not in locals():
    df = events_to_dataframe(events)

df = df.copy()
bucket_edges = [bucket[0] for bucket in BUCKETS] + [BUCKETS[-1][1]]
bucket_labels = [bucket[2] for bucket in BUCKETS]
df['bucket'] = pd.cut(df['ratio'], bins=bucket_edges, labels=bucket_labels, right=False, include_lowest=True)

def _filtered_primary_groups(subset_records):
    available = {event['primary'] for event in subset_records}
    groups = {}
    for name, members in PRIMARY_GROUPS.items():
        members_in_subset = [cat for cat in members if cat in available]
        if members_in_subset:
            groups[name] = members_in_subset
    if not groups:
        for cat in sorted(available):
            groups[cat] = [cat]
    return groups

def _build_special_rows(subset_records):
    specials = []
    special_specs = [
        ("All-In", lambda event: event.get("is_all_in")),
        ("1 BB", lambda event: event.get("is_one_bb")),
    ]
    for label, predicate in special_specs:
        filtered = [event for event in subset_records if predicate(event)]
        if not filtered:
            continue
        groups = _filtered_primary_groups(filtered)
        specials.extend(
            summarize_events(
                filtered,
                [(0.0, float("inf"), label)],
                groups,
                DRAW_FLAGS,
            )
        )
    return specials

def display_heatmap_for_mask(mask, title, base_color='#1f77b4', group_color='#2ca25f'):
    subset = df.loc[mask]
    subset_records = subset.to_dict('records')
    if not subset_records:
        display(pd.DataFrame({'Bucket': bucket_labels, 'Message': 'No events'}))
        return
    groups = _filtered_primary_groups(subset_records)
    summary_subset = summarize_events(subset_records, BUCKETS, groups, DRAW_FLAGS)
    special_rows = _build_special_rows(subset_records)
    display_heatmap_tables(
        summary_subset,
        title,
        base_color=base_color,
        group_color=group_color,
        special_rows=special_rows,
    )


In [7]:
summary_records = summarize_events(events, BUCKETS, PRIMARY_GROUPS, DRAW_FLAGS)
special_rows = _build_special_rows(events)
display_heatmap_tables(summary_records, "All C-Bets", special_rows=special_rows)


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,410.0,2367.0,1760.0,741.0,401.0,33.0,135.0,189.0,140.0
Air,48.3,48.0,44.0,37.8,31.9,48.5,40.7,33.9,52.9
Underpair,10.7,10.1,10.5,7.6,7.5,6.1,7.4,11.1,10.0
Bottom Pair,3.2,2.5,2.7,2.8,2.5,3.0,3.0,1.6,5.7
Middle Pair,6.1,5.6,7.3,3.2,7.2,9.1,3.0,5.3,9.3
Top Pair,12.9,16.5,16.1,20.9,21.2,9.1,18.5,17.5,10.0
Overpair,4.9,4.8,6.8,13.2,14.7,15.2,13.3,14.8,1.4
Two Pair,7.6,7.3,8.8,9.4,10.2,6.1,9.6,10.1,5.0
Trips/Set,5.6,4.2,2.7,3.8,3.5,3.0,2.2,4.2,4.3
Straight,0.0,0.5,0.9,0.9,0.2,0.0,0.7,0.5,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,410.0,2367.0,1760.0,741.0,401.0,33.0,135.0,189.0,140.0
Air,48.3,48.0,44.0,37.8,31.9,48.5,40.7,33.9,52.9
Weak Pair,20.0,18.1,20.5,13.6,17.2,18.2,13.3,18.0,25.0
Top Pair,12.9,16.5,16.1,20.9,21.2,9.1,18.5,17.5,10.0
Overpair,4.9,4.8,6.8,13.2,14.7,15.2,13.3,14.8,1.4
Two Pair,7.6,7.3,8.8,9.4,10.2,6.1,9.6,10.1,5.0
Trips/Set,5.6,4.2,2.7,3.8,3.5,3.0,2.2,4.2,4.3
Monster,0.7,1.2,1.2,1.2,1.2,0.0,2.2,1.6,1.4
Draw,11.7,10.6,12.2,11.2,11.5,12.1,13.3,12.7,14.3


In [8]:
display_heatmap_for_mask(df['in_position'], 'C-Bets In Position', base_color='#1f77b4', group_color='#2ca25f')
display_heatmap_for_mask(~df['in_position'], 'C-Bets Out of Position', base_color='#d95f02', group_color='#7570b3')


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,231.0,1592.0,1144.0,490.0,238.0,23.0,85.0,122.0,66.0
Air,48.1,47.6,42.9,36.1,28.6,47.8,41.2,29.5,50.0
Underpair,11.3,10.2,10.6,8.2,8.0,4.3,8.2,14.8,12.1
Bottom Pair,2.2,2.7,2.7,2.9,2.1,4.3,2.4,1.6,6.1
Middle Pair,6.1,6.2,7.7,3.7,8.4,8.7,1.2,4.9,9.1
Top Pair,13.4,17.0,16.2,23.7,21.8,4.3,14.1,12.3,9.1
Overpair,4.8,4.0,7.0,12.2,13.9,17.4,18.8,18.9,nan
Two Pair,7.8,7.0,8.9,9.6,11.8,8.7,8.2,9.8,7.6
Trips/Set,5.6,4.0,2.7,2.9,4.2,4.3,3.5,6.6,3.0
Straight,0.0,0.6,0.9,0.8,0.4,0.0,1.2,0.8,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,231.0,1592.0,1144.0,490.0,238.0,23.0,85.0,122.0,66.0
Air,48.1,47.6,42.9,36.1,28.6,47.8,41.2,29.5,50.0
Weak Pair,19.5,19.2,21.0,14.7,18.5,17.4,11.8,21.3,27.3
Top Pair,13.4,17.0,16.2,23.7,21.8,4.3,14.1,12.3,9.1
Overpair,4.8,4.0,7.0,12.2,13.9,17.4,18.8,18.9,0.0
Two Pair,7.8,7.0,8.9,9.6,11.8,8.7,8.2,9.8,7.6
Trips/Set,5.6,4.0,2.7,2.9,4.2,4.3,3.5,6.6,3.0
Monster,0.9,1.3,1.3,0.8,1.3,0.0,2.4,1.6,3.0
Draw,13.0,10.4,12.7,11.6,10.5,4.3,15.3,12.3,15.2


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,179.0,775.0,616.0,251.0,163.0,10.0,50.0,67.0,74.0
Air,48.6,48.6,46.1,41.0,36.8,50.0,40.0,41.8,55.4
Underpair,10.1,9.7,10.2,6.4,6.7,10.0,6.0,4.5,8.1
Bottom Pair,4.5,1.9,2.8,2.8,3.1,0.0,4.0,1.5,5.4
Middle Pair,6.1,4.3,6.5,2.4,5.5,10.0,6.0,6.0,9.5
Top Pair,12.3,15.5,16.1,15.5,20.2,20.0,26.0,26.9,10.8
Overpair,5.0,6.3,6.3,15.1,16.0,10.0,4.0,7.5,2.7
Two Pair,7.3,7.7,8.4,9.2,8.0,0.0,12.0,10.4,2.7
Trips/Set,5.6,4.8,2.6,5.6,2.5,0.0,0.0,nan,5.4
Straight,0.0,0.4,0.8,1.2,0.0,0.0,0.0,nan,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,179.0,775.0,616.0,251.0,163.0,10.0,50.0,67.0,74.0
Air,48.6,48.6,46.1,41.0,36.8,50.0,40.0,41.8,55.4
Weak Pair,20.7,15.9,19.5,11.6,15.3,20.0,16.0,11.9,23.0
Top Pair,12.3,15.5,16.1,15.5,20.2,20.0,26.0,26.9,10.8
Overpair,5.0,6.3,6.3,15.1,16.0,10.0,4.0,7.5,2.7
Two Pair,7.3,7.7,8.4,9.2,8.0,0.0,12.0,10.4,2.7
Trips/Set,5.6,4.8,2.6,5.6,2.5,0.0,0.0,0.0,5.4
Monster,0.6,1.2,1.0,2.0,1.2,0.0,2.0,1.5,0.0
Draw,10.1,11.2,11.4,10.4,12.9,30.0,10.0,13.4,13.5


In [9]:

def flop_group(count):
    if count <= 2:
        return '2 Players'
    if count == 3:
        return '3 Players'
    return '4+ Players'

df['flop_group'] = df['flop_players'].apply(flop_group)

cbet_group_titles = [
    ('2 Players', 'C-Bets (2 Players)'),
    ('3 Players', 'C-Bets (3 Players)'),
    ('4+ Players', 'C-Bets (4 Players)'),
]

for group_key, title in cbet_group_titles:
    mask = df['flop_group'] == group_key
    if mask.any():
        display_heatmap_for_mask(mask, title)


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,318.0,2049.0,1442.0,612.0,317.0,24.0,96.0,123.0,112.0
Air,48.1,49.6,45.2,39.9,34.4,54.2,43.8,37.4,52.7
Underpair,11.6,10.3,10.2,7.8,7.9,8.3,5.2,10.6,9.8
Bottom Pair,3.8,2.5,3.2,3.1,2.2,0.0,4.2,2.4,6.2
Middle Pair,6.0,5.4,7.3,3.4,6.0,8.3,4.2,4.9,9.8
Top Pair,11.6,15.5,15.4,19.8,21.5,12.5,17.7,17.1,8.0
Overpair,5.7,4.2,6.2,12.3,14.5,8.3,11.5,12.2,1.8
Two Pair,6.3,6.9,8.9,9.0,9.5,8.3,11.5,10.6,5.4
Trips/Set,6.3,4.2,2.4,3.6,2.5,0.0,0.0,3.3,5.4
Straight,0.0,0.5,0.8,0.8,0.3,0.0,1.0,0.8,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,318.0,2049.0,1442.0,612.0,317.0,24.0,96.0,123.0,112.0
Air,48.1,49.6,45.2,39.9,34.4,54.2,43.8,37.4,52.7
Weak Pair,21.4,18.3,20.7,14.4,16.1,16.7,13.5,17.9,25.9
Top Pair,11.6,15.5,15.4,19.8,21.5,12.5,17.7,17.1,8.0
Overpair,5.7,4.2,6.2,12.3,14.5,8.3,11.5,12.2,1.8
Two Pair,6.3,6.9,8.9,9.0,9.5,8.3,11.5,10.6,5.4
Trips/Set,6.3,4.2,2.4,3.6,2.5,0.0,0.0,3.3,5.4
Monster,0.6,1.3,1.1,1.1,1.6,0.0,2.1,1.6,0.9
Draw,9.7,10.5,11.7,12.3,12.3,12.5,12.5,12.2,13.4


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,72.0,286.0,277.0,113.0,75.0,9.0,27.0,49.0,22.0
Air,50.0,38.8,38.6,28.3,21.3,33.3,33.3,28.6,54.5
Underpair,8.3,8.0,11.6,7.1,6.7,0.0,14.8,14.3,9.1
Bottom Pair,1.4,2.1,0.7,0.9,2.7,11.1,0.0,nan,4.5
Middle Pair,2.8,6.3,6.5,2.7,13.3,11.1,0.0,8.2,4.5
Top Pair,20.8,22.0,19.5,27.4,18.7,0.0,18.5,16.3,18.2
Overpair,2.8,9.1,10.1,15.9,14.7,33.3,14.8,16.3,nan
Two Pair,9.7,8.7,8.3,12.4,14.7,0.0,3.7,6.1,4.5
Trips/Set,2.8,4.2,2.9,3.5,8.0,11.1,11.1,8.2,nan
Straight,0.0,0.3,1.1,1.8,0.0,0.0,0.0,nan,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,72.0,286.0,277.0,113.0,75.0,9.0,27.0,49.0,22.0
Air,50.0,38.8,38.6,28.3,21.3,33.3,33.3,28.6,54.5
Weak Pair,12.5,16.4,18.8,10.6,22.7,22.2,14.8,22.4,18.2
Top Pair,20.8,22.0,19.5,27.4,18.7,0.0,18.5,16.3,18.2
Overpair,2.8,9.1,10.1,15.9,14.7,33.3,14.8,16.3,0.0
Two Pair,9.7,8.7,8.3,12.4,14.7,0.0,3.7,6.1,4.5
Trips/Set,2.8,4.2,2.9,3.5,8.0,11.1,11.1,8.2,0.0
Monster,1.4,0.7,1.8,1.8,0.0,0.0,3.7,2.0,4.5
Draw,19.4,11.5,13.7,7.1,6.7,11.1,14.8,14.3,18.2


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,20.0,32.0,41.0,16.0,9.0,0.0,12.0,17.0,6.0
Air,45.0,25.0,39.0,25.0,33.3,0.0,33.3,23.5,50.0
Underpair,5.0,9.4,12.2,0.0,0.0,0.0,8.3,5.9,16.7
Bottom Pair,0.0,0.0,0.0,6.2,11.1,0.0,0.0,nan,nan
Middle Pair,20.0,9.4,12.2,0.0,0.0,0.0,0.0,nan,16.7
Top Pair,5.0,31.2,19.5,18.8,33.3,0.0,25.0,23.5,16.7
Overpair,0.0,3.1,2.4,31.2,22.2,0.0,25.0,29.4,nan
Two Pair,20.0,18.8,4.9,6.2,0.0,0.0,8.3,17.6,nan
Trips/Set,5.0,3.1,9.8,12.5,0.0,0.0,0.0,nan,nan
Flush Draw,10.0,9.4,12.2,0.0,22.2,0.0,16.7,11.8,16.7


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,20.0,32.0,41.0,16.0,9.0,0.0,12.0,17.0,6.0
Air,45.0,25.0,39.0,25.0,33.3,0.0,33.3,23.5,50.0
Weak Pair,25.0,18.8,24.4,6.2,11.1,0.0,8.3,5.9,33.3
Top Pair,5.0,31.2,19.5,18.8,33.3,0.0,25.0,23.5,16.7
Overpair,0.0,3.1,2.4,31.2,22.2,0.0,25.0,29.4,0.0
Two Pair,20.0,18.8,4.9,6.2,0.0,0.0,8.3,17.6,0.0
Trips/Set,5.0,3.1,9.8,12.5,0.0,0.0,0.0,0.0,0.0
Draw,15.0,9.4,22.0,0.0,22.2,0.0,16.7,11.8,16.7


In [10]:

response_rows = response_events(events)
responses_df = pd.DataFrame(response_rows)

def _prepare_response_table(records):
    if not records:
        return pd.DataFrame()
    df = pd.DataFrame(records)
    if 'Bucket' not in df.columns:
        return pd.DataFrame()
    df = df.set_index('Bucket').T.apply(pd.to_numeric, errors='coerce')
    row_order = ['Events', 'Fold', 'Call', 'Raise', 'Continue']
    ordered = [row for row in row_order if row in df.index]
    extras = [row for row in df.index if row not in row_order]
    if ordered or extras:
        df = df.loc[ordered + extras]
    return df

def _response_summary(subset_df):
    if subset_df is None or subset_df.empty:
        return {'Events': 0, 'Fold': 0.0, 'Call': 0.0, 'Raise': 0.0, 'Continue': 0.0}
    events_count = len(subset_df)
    response_counts = subset_df['response'].value_counts()
    continue_count = int(response_counts.get('Call', 0) + response_counts.get('Raise', 0))
    return {
        'Events': events_count,
        'Fold': response_counts.get('Fold', 0) / events_count * 100,
        'Call': response_counts.get('Call', 0) / events_count * 100,
        'Raise': response_counts.get('Raise', 0) / events_count * 100,
        'Continue': continue_count / events_count * 100,
    }

def _compute_special_columns(source_df):
    specials = {}
    if source_df is None:
        return specials
    if 'is_all_in_cbet' in source_df.columns:
        specials['All-In'] = _response_summary(source_df[source_df['is_all_in_cbet']])
    if 'is_one_bb_cbet' in source_df.columns:
        specials['1 BB'] = _response_summary(source_df[source_df['is_one_bb_cbet']])
    return specials

def _style_response_table(df, title, base_color: str = '#6baed6'):
    if df.empty:
        return None
    percent_rows = [row for row in ['Fold', 'Call', 'Raise', 'Continue'] if row in df.index]
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
        styled = styled.background_gradient(
            cmap=_gradient_cmap(base_color),
            axis=None,
            subset=pd.IndexSlice[percent_rows, :]
        )
    special_cols = [col for col in ['All-In', '1 BB'] if col in df.columns]
    if special_cols:
        first = special_cols[0]
        idx = df.columns.get_loc(first)
        styled = styled.set_table_styles([
            {'selector': f'th.col_heading.level0.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
            {'selector': f'td.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
        ], overwrite=False)
    return styled.set_caption(title)

def display_response_tables(summary_records, title, base_color='#6baed6', source_df=None):
    df = _prepare_response_table(summary_records)
    if df.empty:
        display(pd.DataFrame({'Bucket': bucket_labels, 'Message': 'No events'}))
        return
    specials = _compute_special_columns(source_df)
    if specials:
        for col_name, values in specials.items():
            for row in df.index:
                df.loc[row, col_name] = values.get(row, 0.0)
    styled = _style_response_table(df, title, base_color=base_color)
    if styled is not None:
        display(styled)

def display_response_for_mask(mask, title, base_color='#6baed6'):
    if responses_df.empty:
        display(pd.DataFrame({'Bucket': bucket_labels, 'Message': 'No events'}))
        return
    subset = responses_df.loc[mask]
    if subset.empty:
        display(pd.DataFrame({'Bucket': bucket_labels, 'Message': 'No events'}))
        return
    summary_subset = summarize_responses_df(subset, BUCKETS)
    display_response_tables(summary_subset, title, base_color=base_color, source_df=subset)

def _prepare_turn_response_records(turn_events, turn_responses, line_type, response_type):
    records = []
    for response in turn_responses:
        if response.get('line_type') != line_type:
            continue
        if response.get('response') != response_type:
            continue
        primary = response.get('responder_primary')
        if not primary:
            continue
        records.append({
            'ratio': float(response.get('ratio', 0.0)),
            'primary': primary,
            'has_flush_draw': bool(response.get('responder_has_flush_draw')),
            'has_oesd_dg': bool(response.get('responder_has_oesd_dg')),
            'made_flush': bool(response.get('responder_made_flush')),
            'made_straight': bool(response.get('responder_made_straight')),
        })
    return records

def _summarize_response_records(records, buckets):
    summary = []
    for low, high, label in buckets:
        bucket_records = [rec for rec in records if low <= rec['ratio'] < high]
        if not bucket_records:
            row = {'Bucket': label, 'Events': 0}
            for group_name in PRIMARY_GROUPS:
                row[group_name] = 0.0
            for draw_name in DRAW_FLAGS:
                row[draw_name] = 0.0
            summary.append(row)
            continue
        total = len(bucket_records)
        row = {'Bucket': label, 'Events': total}
        for group_name, members in PRIMARY_GROUPS.items():
            row[group_name] = sum(rec['primary'] in members for rec in bucket_records) / total * 100
        for draw_name, field in DRAW_FLAGS.items():
            row[draw_name] = sum(bool(rec.get(field)) for rec in bucket_records) / total * 100
        summary.append(row)
    return summary

def display_turn_response_holdings(turn_events, turn_responses, line_type, response_type, title, *, base_color='#1f77b4', group_color='#2ca25f'):
    records = _prepare_turn_response_records(turn_events, turn_responses, line_type, response_type)
    if not records:
        print(f"No {response_type.lower()} responses recorded for {line_type.lower()} line.")
        return
    summary = _summarize_response_records(records, TURN_BUCKETS_TURN)
    display_heatmap_tables(summary, title, base_color=base_color, group_color=group_color)

def _villain_response_metrics(subset_df):
    events = len(subset_df)
    if events == 0:
        return {
            'Events': 0,
            'Fold%': 0.0,
            'Call%': 0.0,
            'Raise%': 0.0,
            'Continue%': 0.0,
            'Avg Call Amount': 0.0,
            'Avg Raise Amount': 0.0,
            'Expected Continue Amount': 0.0,
        }
    fold_pct = subset_df['response'].eq('Fold').mean() * 100
    call_mask = subset_df['response'].eq('Call')
    raise_mask = subset_df['response'].eq('Raise')
    call_pct = call_mask.mean() * 100
    raise_pct = raise_mask.mean() * 100
    continue_pct = call_pct + raise_pct
    avg_call_amt = subset_df.loc[call_mask, 'response_amount'].mean() if call_mask.any() else 0.0
    avg_raise_amt = subset_df.loc[raise_mask, 'response_amount'].mean() if raise_mask.any() else 0.0
    expected_continue = (call_pct / 100) * (avg_call_amt or 0.0) + (raise_pct / 100) * (avg_raise_amt or 0.0)
    return {
        'Events': float(events),
        'Fold%': fold_pct,
        'Call%': call_pct,
        'Raise%': raise_pct,
        'Continue%': continue_pct,
        'Avg Call Amount': avg_call_amt or 0.0,
        'Avg Raise Amount': avg_raise_amt or 0.0,
        'Expected Continue Amount': expected_continue,
    }

def summarize_response_ev(responses_df, buckets):
    summary = []
    for low, high, label in buckets:
        subset = responses_df[(responses_df['ratio'] >= low) & (responses_df['ratio'] < high)]
        metrics = _villain_response_metrics(subset)
        metrics['Bucket'] = label
        summary.append(metrics)
    return summary

def display_response_ev_table(responses_df, title, base_color='#6baed6'):
    records = summarize_response_ev(responses_df, TURN_BUCKETS)
    df = pd.DataFrame(records)
    if df.empty:
        display(pd.DataFrame({'Bucket': [bucket[2] for bucket in TURN_BUCKETS], 'Message': 'No events'}))
        return
    df = df.set_index('Bucket').T.apply(pd.to_numeric, errors='coerce')
    styled = (
        df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[['Fold%', 'Call%', 'Raise%', 'Continue%'], :])
        .format('{:.2f}', subset=pd.IndexSlice[['Avg Call Amount', 'Avg Raise Amount', 'Expected Continue Amount'], :])
        .background_gradient(cmap=_gradient_cmap(base_color), axis=None, subset=pd.IndexSlice[['Fold%', 'Call%', 'Raise%', 'Continue%'], :])
        .set_caption(title)
    )
    display(styled)

if responses_df.empty:
    print("No c-bet response events available.")
else:
    responses_df['bucket'] = pd.cut(
        responses_df['ratio'],
        bins=bucket_edges,
        labels=bucket_labels,
        right=False,
        include_lowest=True,
    )



In [11]:
def _filter_standard_cbet_events(events_df):
    if events_df is None or events_df.empty:
        return events_df.iloc[0:0].copy()
    mask = pd.Series(True, index=events_df.index)
    if 'is_all_in' in events_df.columns:
        mask &= ~events_df['is_all_in']
    if 'is_one_bb' in events_df.columns:
        mask &= ~events_df['is_one_bb']
    return events_df.loc[mask].copy()


def _filter_standard_cbet_responses(responses_df):
    if responses_df is None or responses_df.empty:
        return responses_df.iloc[0:0].copy()
    mask = pd.Series(True, index=responses_df.index)
    if 'is_all_in_cbet' in responses_df.columns:
        mask &= ~responses_df['is_all_in_cbet']
    if 'is_one_bb_cbet' in responses_df.columns:
        mask &= ~responses_df['is_one_bb_cbet']
    return responses_df.loc[mask].copy()


def _safe_mean(series):
    if series is None or len(series) == 0:
        return 0.0
    cleaned = series.dropna()
    if cleaned.empty:
        return 0.0
    value = cleaned.mean()
    return float(value) if pd.notna(value) else 0.0


def _prepare_cbet_event_metrics(events_subset, responses_subset, *, exclude_special=True):
    if events_subset is None or events_subset.empty:
        return pd.DataFrame()
    events_work = events_subset.copy()
    responses_work = responses_subset.copy()
    if exclude_special:
        events_work = _filter_standard_cbet_events(events_work)
        responses_work = _filter_standard_cbet_responses(responses_work)
    if events_work.empty:
        return pd.DataFrame()
    events_work = events_work.rename(columns={'player': 'bettor'})
    base_cols = [
        'hand_number',
        'bettor',
        'ratio',
        'bucket',
        'bet_amount',
        'bet_amount_bb',
        'big_blind',
    ]
    missing = [col for col in base_cols if col not in events_work.columns]
    if missing:
        raise KeyError(f"Missing columns in events dataframe: {', '.join(missing)}")
    events_work = events_work[base_cols].copy()
    if not responses_work.empty:
        allowed = set(zip(events_work['hand_number'], events_work['bettor']))
        responses_work = responses_work[
            responses_work.apply(lambda row: (row['hand_number'], row['bettor']) in allowed, axis=1)
        ]
    if responses_work.empty:
        grouped = pd.DataFrame(
            columns=[
                'hand_number',
                'bettor',
                'continue_amount',
                'call_amount',
                'raise_amount',
                'call_flag',
                'raise_flag',
            ]
        )
    else:
        prepped = responses_work.copy()
        prepped['continue_amount'] = np.where(
            prepped['response'].isin(['Call', 'Raise']),
            prepped['response_amount'],
            0.0,
        )
        prepped['call_amount'] = np.where(
            prepped['response'] == 'Call',
            prepped['response_amount'],
            0.0,
        )
        prepped['raise_amount'] = np.where(
            prepped['response'] == 'Raise',
            prepped['response_amount'],
            0.0,
        )
        prepped['call_flag'] = (prepped['response'] == 'Call').astype(int)
        prepped['raise_flag'] = (prepped['response'] == 'Raise').astype(int)
        grouped = (
            prepped.groupby(['hand_number', 'bettor'], as_index=False)
            .agg({
                'continue_amount': 'sum',
                'call_amount': 'sum',
                'raise_amount': 'sum',
                'call_flag': 'max',
                'raise_flag': 'max',
            })
        )
    metrics = events_work.merge(grouped, on=['hand_number', 'bettor'], how='left')
    for col in ['continue_amount', 'call_amount', 'raise_amount']:
        metrics[col] = metrics[col].fillna(0.0)
    for col in ['call_flag', 'raise_flag']:
        metrics[col] = metrics[col].fillna(0).astype(int)
    metrics['continue_flag'] = metrics['continue_amount'] > 0
    metrics['call_only_flag'] = (metrics['call_flag'] > 0) & (metrics['raise_flag'] == 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        metrics['pot_before'] = np.where(
            metrics['ratio'] > 0,
            metrics['bet_amount'] / metrics['ratio'],
            np.nan,
        )
        metrics['bet_ratio_pct'] = metrics['ratio'].astype(float) * 100.0
        metrics['raise_ratio_pct'] = np.where(
            (metrics['raise_flag'] > 0) & (metrics['pot_before'] > 0),
            (metrics['raise_amount'] / metrics['pot_before']) * 100.0,
            np.nan,
        )
        metrics['breakeven_fold_pct'] = np.where(
            (metrics['bet_amount'] + metrics['pot_before']) > 0,
            metrics['bet_amount'] / (metrics['bet_amount'] + metrics['pot_before']) * 100,
            np.nan,
        )
    return metrics


def _compute_cbet_bucket_metrics(metrics_df, buckets=BUCKETS):
    results = []
    if metrics_df is None or metrics_df.empty:
        for _, _, label in buckets:
            results.append({'Bucket': label, 'Events': 0.0})
        return results
    for low, high, label in buckets:
        bucket_mask = (metrics_df['ratio'] >= low) & (metrics_df['ratio'] < high)
        bucket_events = metrics_df.loc[bucket_mask]
        total = len(bucket_events)
        if total == 0:
            results.append({'Bucket': label, 'Events': 0.0})
            continue
        continue_events = int(bucket_events['continue_flag'].sum())
        raise_events = int(bucket_events['raise_flag'].sum())
        call_only_events = int(bucket_events['call_only_flag'].sum())
        continue_pct = continue_events / total * 100
        raise_pct = raise_events / total * 100
        call_only_pct = call_only_events / total * 100
        fold_pct = 100.0 - continue_pct
        avg_ratio_pct = _safe_mean(bucket_events['bet_ratio_pct'])
        raise_mask = bucket_events['raise_flag'] > 0
        avg_raise_ratio_pct = _safe_mean(bucket_events.loc[raise_mask, 'raise_ratio_pct'])
        avg_breakeven = _safe_mean(bucket_events['breakeven_fold_pct'])
        expected_call_contrib_pct = (call_only_pct * avg_ratio_pct) / 100.0 if not np.isnan(avg_ratio_pct) else 0.0
        expected_raise_contrib_pct = (raise_pct * avg_raise_ratio_pct) / 100.0 if not np.isnan(avg_raise_ratio_pct) else 0.0
        total_contrib_pct = expected_call_contrib_pct + expected_raise_contrib_pct
        row = {
            'Bucket': label,
            'Events': float(total),
            'Fold%': fold_pct,
            'Continue%': continue_pct,
            'Call-Only%': call_only_pct,
            'Raise%': raise_pct,
            'Avg Size (Pot%)': avg_ratio_pct,
            'Avg Raise Size (Pot%)': avg_raise_ratio_pct,
            'Expected Call Contribution (Pot%)': expected_call_contrib_pct,
            'Expected Raise Contribution (Pot%)': expected_raise_contrib_pct,
            'Raise Contribution Total (Pot%)': total_contrib_pct,
            'Breakeven Fold%': avg_breakeven,
        }
        row['Fold Surplus'] = row['Fold%'] - row['Breakeven Fold%']
        results.append(row)
    return results


VALUE_COLUMNS_CBET = [
    'Bucket',
    'Events',
    'Continue%',
    'Call-Only%',
    'Raise%',
    'Avg Size (Pot%)',
    'Avg Raise Size (Pot%)',
    'Expected Call Contribution (Pot%)',
    'Expected Raise Contribution (Pot%)',
    'Raise Contribution Total (Pot%)',
]


PRESSURE_COLUMNS_CBET = [
    'Bucket',
    'Events',
    'Fold%',
    'Avg Size (Pot%)',
    'Breakeven Fold%',
    'Fold Surplus',
]


_FOLD_SURPLUS_CMAP = LinearSegmentedColormap.from_list(
    'fold_surplus_cmp_flop', ['#b91c1c', '#ffffff', '#15803d']
)


def _style_cbet_value_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    percent_rows = [row for row in df.index if row != 'Events']
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
        contribution_rows = [row for row in percent_rows if 'Contribution' in row]
        size_rows = [row for row in percent_rows if 'Size' in row]
        pct_rows_other = [row for row in percent_rows if row not in contribution_rows + size_rows]
        if pct_rows_other:
            styled = styled.background_gradient(
                cmap='YlOrRd',
                axis=None,
                subset=pd.IndexSlice[pct_rows_other, :],
            )
        if size_rows:
            styled = styled.background_gradient(
                cmap='BuGn',
                axis=None,
                subset=pd.IndexSlice[size_rows, :],
            )
        if contribution_rows:
            styled = styled.background_gradient(
                cmap='PuBu',
                axis=None,
                subset=pd.IndexSlice[contribution_rows, :],
            )
    return styled.set_caption(title)


def _style_cbet_pressure_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    desired_order = ['Events', 'Fold%', 'Avg Size (Pot%)', 'Breakeven Fold%', 'Fold Surplus']
    existing_order = [row for row in desired_order if row in df.index]
    df = df.loc[existing_order]
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    percent_rows = [row for row in df.index if row != 'Events']
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
    if 'Fold%' in df.index:
        styled = styled.background_gradient(
            cmap='YlOrRd',
            axis=None,
            subset=pd.IndexSlice[['Fold%'], :],
        )
    if 'Avg Size (Pot%)' in df.index:
        styled = styled.background_gradient(
            cmap='BuGn',
            axis=None,
            subset=pd.IndexSlice[['Avg Size (Pot%)'], :],
        )
    if 'Fold Surplus' in df.index:
        fold_values = df.loc['Fold Surplus'].dropna().astype(float)
        if not fold_values.empty:
            max_abs = max(abs(fold_values.min()), abs(fold_values.max()))
            if max_abs == 0:
                max_abs = 1.0
            styled = styled.background_gradient(
                cmap=_FOLD_SURPLUS_CMAP,
                axis=None,
                subset=pd.IndexSlice[['Fold Surplus'], :],
                vmin=-max_abs,
                vmax=max_abs,
            )
    return styled.set_caption(title)


def _display_cbet_event_tables(metrics_df, title_prefix):
    if metrics_df is None or metrics_df.empty:
        print(f'No events available for {title_prefix}.')
        return
    records = _compute_cbet_bucket_metrics(metrics_df)
    if not records:
        print(f'No bucketed metrics available for {title_prefix}.')
        return
    metrics_table = pd.DataFrame(records)
    value_columns = [col for col in VALUE_COLUMNS_CBET if col in metrics_table.columns]
    pressure_columns = [col for col in PRESSURE_COLUMNS_CBET if col in metrics_table.columns]
    value_table = metrics_table[value_columns]
    pressure_table = metrics_table[pressure_columns]
    value_styled = _style_cbet_value_table(value_table, f"{title_prefix} - Value Profile")
    if value_styled is not None:
        display(value_styled)
    pressure_styled = _style_cbet_pressure_table(pressure_table, f"{title_prefix} - Fold Pressure")
    if pressure_styled is not None:
        display(pressure_styled)


def display_cbet_event_ev_tables(title_prefix='Flop C-Bet Event Outcomes', *, exclude_special=True):
    if df.empty or responses_df.empty:
        print('No c-bet events or responses available for event-level contribution tables.')
        return
    base_metrics = _prepare_cbet_event_metrics(df, responses_df, exclude_special=exclude_special)
    if base_metrics.empty:
        print('No c-bet events remain after filtering special cases.')
        return
    subsets = [
        (base_metrics, f"{title_prefix} (Overall)"),
        (
            _prepare_cbet_event_metrics(
                df[df['in_position']],
                responses_df[responses_df['bettor_in_position']],
                exclude_special=exclude_special,
            ),
            f"{title_prefix} (In Position)",
        ),
        (
            _prepare_cbet_event_metrics(
                df[~df['in_position']],
                responses_df[~responses_df['bettor_in_position']],
                exclude_special=exclude_special,
            ),
            f"{title_prefix} (Out of Position)",
        ),
    ]
    for metrics_slice, label in subsets:
        if metrics_slice is None or metrics_slice.empty:
            continue
        _display_cbet_event_tables(metrics_slice, label)


### Flop Event-Level Outcomes

These tables mirror the turn explorer value/fold summaries.
They exclude one big blind probes and all-in bets, summarising villain
contributions in pot-percent terms for easier sizing comparisons.


In [12]:
if df.empty or responses_df.empty:
    print('No flop events or responses available for event-level analysis.')
else:
    display_cbet_event_ev_tables()


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,256,2361,1744,728,387,19,23
Continue%,80.9%,68.4%,62.6%,55.2%,47.0%,42.1%,21.7%
Call-Only%,62.5%,58.2%,52.5%,45.7%,33.6%,36.8%,13.0%
Raise%,18.4%,10.2%,10.1%,9.5%,13.4%,5.3%,8.7%
Avg Size (Pot%),21.5%,31.5%,47.2%,68.5%,91.7%,109.7%,200.4%
Avg Raise Size (Pot%),98.3%,118.0%,190.5%,195.7%,285.4%,200.0%,563.4%
Expected Call Contribution (Pot%),13.5%,18.4%,24.8%,31.3%,30.8%,40.4%,26.1%
Expected Raise Contribution (Pot%),18.0%,12.0%,19.3%,18.5%,38.4%,10.5%,49.0%
Raise Contribution Total (Pot%),31.5%,30.3%,44.1%,49.9%,69.1%,50.9%,75.1%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,256,2361,1744,728,387,19,23
Fold%,19.1%,31.6%,37.4%,44.8%,53.0%,57.9%,78.3%
Avg Size (Pot%),21.5%,31.5%,47.2%,68.5%,91.7%,109.7%,200.4%
Breakeven Fold%,17.6%,23.9%,32.0%,40.6%,47.8%,52.3%,64.8%
Fold Surplus,1.5%,7.7%,5.4%,4.2%,5.2%,5.6%,13.5%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,152,1587,1133,481,228,14,20
Continue%,80.9%,67.2%,62.0%,51.8%,44.3%,21.4%,25.0%
Call-Only%,64.5%,58.7%,52.1%,44.7%,34.2%,14.3%,15.0%
Raise%,16.4%,8.4%,9.9%,7.1%,10.1%,7.1%,10.0%
Avg Size (Pot%),22.1%,31.8%,47.6%,69.3%,92.8%,110.2%,184.1%
Avg Raise Size (Pot%),110.4%,117.2%,210.1%,191.9%,353.0%,200.0%,563.4%
Expected Call Contribution (Pot%),14.3%,18.7%,24.8%,31.0%,31.7%,15.7%,27.6%
Expected Raise Contribution (Pot%),18.2%,9.9%,20.8%,13.6%,35.6%,14.3%,56.3%
Raise Contribution Total (Pot%),32.4%,28.6%,45.6%,44.5%,67.4%,30.0%,84.0%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,152,1587,1133,481,228,14,20
Fold%,19.1%,32.8%,38.0%,48.2%,55.7%,78.6%,75.0%
Avg Size (Pot%),22.1%,31.8%,47.6%,69.3%,92.8%,110.2%,184.1%
Breakeven Fold%,18.1%,24.1%,32.2%,40.9%,48.1%,52.4%,63.4%
Fold Surplus,1.0%,8.7%,5.8%,7.4%,7.6%,26.2%,11.6%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,104,774,611,247,159,5,3
Continue%,80.8%,70.9%,63.8%,61.9%,50.9%,100.0%,0.0%
Call-Only%,59.6%,57.2%,53.2%,47.8%,32.7%,100.0%,0.0%
Raise%,21.2%,13.7%,10.6%,14.2%,18.2%,0.0%,0.0%
Avg Size (Pot%),20.7%,30.8%,46.3%,66.9%,90.1%,108.1%,309.2%
Avg Raise Size (Pot%),84.6%,119.0%,156.7%,199.4%,231.8%,0.0%,0.0%
Expected Call Contribution (Pot%),12.3%,17.6%,24.6%,32.0%,29.5%,108.1%,0.0%
Expected Raise Contribution (Pot%),17.9%,16.3%,16.7%,28.3%,42.3%,0.0%,0.0%
Raise Contribution Total (Pot%),30.2%,33.9%,41.3%,60.2%,71.7%,108.1%,0.0%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,104,774,611,247,159,5,3
Fold%,19.2%,29.1%,36.2%,38.1%,49.1%,0.0%,100.0%
Avg Size (Pot%),20.7%,30.8%,46.3%,66.9%,90.1%,108.1%,309.2%
Breakeven Fold%,17.0%,23.5%,31.6%,40.0%,47.4%,51.9%,74.2%
Fold Surplus,2.2%,5.5%,4.5%,-2.0%,1.7%,-51.9%,25.8%


In [13]:
if not responses_df.empty:
    summary_responses = summarize_responses_df(responses_df, BUCKETS)
    display_response_tables(summary_responses, "Responses to C-Bets", source_df=responses_df)



Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,509,2714,2118,885,493,39,181,236,175
Fold,27.3%,37.5%,44.6%,51.3%,59.0%,56.4%,58.6%,52.1%,22.9%
Call,58.2%,53.6%,46.8%,40.7%,29.2%,35.9%,18.8%,26.7%,61.7%
Raise,14.5%,8.9%,8.5%,8.0%,11.8%,7.7%,22.7%,21.2%,15.4%
Continue,72.7%,62.5%,55.4%,48.7%,41.0%,43.6%,41.4%,47.9%,77.1%


In [14]:
if not responses_df.empty:
    display_response_for_mask(responses_df['bettor_in_position'], 'Responses vs In-Position C-Bets')
    display_response_for_mask(~responses_df['bettor_in_position'], 'Responses vs Out-of-Position C-Bets')


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,295,1890,1448,609,302,29,123,157,86
Fold,29.2%,40.3%,46.8%,55.8%,61.6%,69.0%,59.3%,52.2%,26.7%
Call,59.7%,52.6%,45.2%,38.3%,29.1%,24.1%,22.0%,28.7%,64.0%
Raise,11.2%,7.1%,7.9%,5.9%,9.3%,6.9%,18.7%,19.1%,9.3%
Continue,70.8%,59.7%,53.2%,44.2%,38.4%,31.0%,40.7%,47.8%,73.3%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,214,824,670,276,191,10,58,79,89
Fold,24.8%,31.3%,39.9%,41.3%,55.0%,20.0%,56.9%,51.9%,19.1%
Call,56.1%,55.8%,50.3%,46.0%,29.3%,70.0%,12.1%,22.8%,59.6%
Raise,19.2%,12.9%,9.9%,12.7%,15.7%,10.0%,31.0%,25.3%,21.3%
Continue,75.2%,68.7%,60.1%,58.7%,45.0%,80.0%,43.1%,48.1%,80.9%


In [15]:

if not responses_df.empty:
    def response_flop_group(count):
        if count <= 2:
            return '2 Players'
        if count == 3:
            return '3 Players'
        return '4+ Players'

    responses_df['flop_group'] = responses_df['flop_players'].apply(response_flop_group)

    response_group_titles = [
        ('2 Players', 'Responses (2 Players)'),
        ('3 Players', 'Responses (3 Players)'),
        ('4+ Players', 'Responses (4 Players)'),
    ]

    for group_key, title in response_group_titles:
        mask = responses_df['flop_group'] == group_key
        if mask.any():
            display_response_for_mask(mask, title)


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,309,2048,1441,611,317,24,96,111,112
Fold,22.3%,33.3%,39.7%,45.3%,55.8%,50.0%,53.1%,42.3%,21.4%
Call,61.5%,57.1%,51.0%,45.3%,30.9%,41.7%,21.9%,34.2%,63.4%
Raise,16.2%,9.6%,9.3%,9.3%,13.2%,8.3%,25.0%,23.4%,15.2%
Continue,77.7%,66.7%,60.3%,54.7%,44.2%,50.0%,46.9%,57.7%,78.6%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,141,570,554,224,149,15,49,82,44
Fold,35.5%,49.3%,52.9%,63.8%,65.8%,66.7%,61.2%,56.1%,34.1%
Call,52.5%,43.7%,39.7%,30.8%,25.5%,26.7%,12.2%,22.0%,47.7%
Raise,12.1%,7.0%,7.4%,5.4%,8.7%,6.7%,26.5%,22.0%,18.2%
Continue,64.5%,50.7%,47.1%,36.2%,34.2%,33.3%,38.8%,43.9%,65.9%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,59,96,123,50,27,0,36,43,19
Fold,33.9%,58.3%,65.0%,68.0%,59.3%,0.0%,69.4%,69.8%,5.3%
Call,54.2%,37.5%,30.1%,28.0%,29.6%,0.0%,19.4%,16.3%,84.2%
Raise,11.9%,4.2%,4.9%,4.0%,11.1%,0.0%,11.1%,14.0%,10.5%
Continue,66.1%,41.7%,35.0%,32.0%,40.7%,0.0%,30.6%,30.2%,94.7%


In [16]:
if not responses_df.empty:
    raise_rows = responses_df[responses_df['response'] == 'Raise']
    if raise_rows.empty:
        print('No raise responses recorded.')
    else:
        raise_records = []
        for _, row in raise_rows.iterrows():
            primary = row.get('responder_primary')
            if not primary:
                continue
            raise_records.append(
                {
                    'ratio': float(row['ratio']),
                    'primary': primary,
                    'has_flush_draw': bool(row.get('responder_has_flush_draw')),
                    'has_oesd_dg': bool(row.get('responder_has_oesd_dg')),
                }
            )
        if raise_records:
            raise_primary_groups = {cat: [cat] for cat in available_primary_categories(raise_records)}
            raise_draw_flags = {
                'Flush Draw': 'has_flush_draw',
                'OESD/DG': 'has_oesd_dg',
            }
            raise_summary = summarize_events(raise_records, BUCKETS, raise_primary_groups, raise_draw_flags)
            print("Opponent holdings when they raise the hero's c-bet (bucketed by hero's sizing).")
            display_heatmap_tables(
                raise_summary,
                'Opponent Raise Holdings vs Hero C-Bet',
                base_color='#cb181d',
                group_color='#88419d',
            )
        else:
            print('Raise responses lack classified hands.')
    call_rows = responses_df[responses_df['response'] == 'Call']
    if call_rows.empty:
        print('No call responses recorded.')
    else:
        call_records = []
        for _, row in call_rows.iterrows():
            primary = row.get('responder_primary')
            if not primary:
                continue
            call_records.append(
                {
                    'ratio': float(row['ratio']),
                    'primary': primary,
                    'has_flush_draw': bool(row.get('responder_has_flush_draw')),
                    'has_oesd_dg': bool(row.get('responder_has_oesd_dg')),
                }
            )
        if call_records:
            call_primary_groups = {cat: [cat] for cat in available_primary_categories(call_records)}
            call_draw_flags = {
                'Flush Draw': 'has_flush_draw',
                'OESD/DG': 'has_oesd_dg',
            }
            call_summary = summarize_events(call_records, BUCKETS, call_primary_groups, call_draw_flags)
            print("Opponent holdings when they call the hero's c-bet (bucketed by hero's sizing).")
            display_heatmap_tables(
                call_summary,
                'Opponent Call Holdings vs Hero C-Bet',
                base_color='#2171b5',
                group_color='#6baed6',
            )
        else:
            print('Call responses lack classified hands.')

    if turn_events:
        display_turn_response_holdings(turn_events, turn_responses, 'Barrel', 'Raise', 'Opponent Raise Holdings vs Hero Barrel', base_color='#cb181d', group_color='#88419d')
        display_turn_response_holdings(turn_events, turn_responses, 'Delayed', 'Raise', 'Opponent Raise Holdings vs Hero Delayed C-Bet', base_color='#cb181d', group_color='#88419d')
        display_turn_response_holdings(turn_events, turn_responses, 'Barrel', 'Call', 'Opponent Call Holdings vs Hero Barrel', base_color='#2171b5', group_color='#6baed6')
        display_turn_response_holdings(turn_events, turn_responses, 'Delayed', 'Call', 'Opponent Call Holdings vs Hero Delayed C-Bet', base_color='#2171b5', group_color='#6baed6')
else:
    print('No c-bet response events available.')



Opponent holdings when they raise the hero's c-bet (bucketed by hero's sizing).


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,74.0,241.0,181.0,71.0,58.0,3.0,41.0
Air,41.9,36.5,28.2,28.2,29.3,66.7,39.0
Bottom Pair,1.4,1.2,5.0,4.2,6.9,0.0,7.3
Flush,2.7,0.8,0.0,1.4,0.0,0.0,0.0
Full House,0.0,1.2,1.1,0.0,0.0,0.0,0.0
Middle Pair,10.8,4.1,2.8,7.0,6.9,0.0,9.8
Overpair,1.4,2.5,4.4,8.5,5.2,0.0,4.9
Quads,0.0,0.0,0.0,0.0,1.7,0.0,0.0
Straight,1.4,1.2,2.2,2.8,1.7,0.0,0.0
Top Pair,21.6,24.5,24.3,23.9,22.4,0.0,22.0


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,74.0,241.0,181.0,71.0,58.0,3.0,41.0
Air,41.9,36.5,28.2,28.2,29.3,66.7,39.0
Weak Pair,16.2,10.0,13.8,14.1,19.0,0.0,19.5
Top Pair,21.6,24.5,24.3,23.9,22.4,0.0,22.0
Overpair,1.4,2.5,4.4,8.5,5.2,0.0,4.9
Two Pair,9.5,15.4,16.0,15.5,15.5,0.0,4.9
Trips/Set,5.4,7.9,9.9,5.6,5.2,33.3,9.8
Monster,4.1,3.3,3.3,4.2,3.4,0.0,0.0
Draw,27.0,10.8,14.9,14.1,17.2,66.7,22.0


Opponent holdings when they call the hero's c-bet (bucketed by hero's sizing).


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,296.0,1454.0,992.0,360.0,144.0,14.0,34.0
Air,58.1,45.7,44.8,45.6,29.2,50.0,35.3
Bottom Pair,6.8,6.5,6.9,7.8,8.3,0.0,2.9
Flush,0.7,0.2,0.7,0.0,0.7,0.0,0.0
Full House,0.3,0.3,0.4,0.3,0.0,0.0,0.0
Middle Pair,4.7,12.1,13.6,11.9,10.4,7.1,11.8
Overpair,0.7,0.4,1.5,0.8,6.9,0.0,8.8
Quads,0.7,0.2,0.1,0.0,0.7,0.0,0.0
Straight,0.7,0.3,0.7,0.6,0.7,0.0,0.0
Top Pair,10.1,14.2,14.3,19.4,22.2,28.6,11.8


...

Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,31.0,60.0,75.0,35.0,21.0,1.0,13.0
Air,29.0,28.3,22.7,31.4,33.3,0.0,30.8
Weak Pair,3.2,11.7,8.0,14.3,4.8,100.0,7.7
Top Pair,29.0,15.0,17.3,11.4,4.8,0.0,7.7
Overpair,0.0,0.0,4.0,0.0,0.0,0.0,0.0
Two Pair,12.9,10.0,14.7,8.6,33.3,0.0,15.4
Trips/Set,9.7,15.0,13.3,22.9,9.5,0.0,7.7
Monster,16.1,20.0,20.0,11.4,14.3,0.0,30.8
Draw,22.6,23.3,17.3,22.9,14.3,0.0,23.1


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,265.0,229.0,497.0,403.0,158.0,19.0,29.0
Air,41.5,41.5,32.2,30.8,31.6,10.5,10.3
Underpair,10.9,4.4,4.0,3.7,3.8,0.0,6.9
Bottom Pair,4.9,3.1,5.4,6.7,7.6,5.3,6.9
Middle Pair,8.3,10.9,9.9,11.7,5.7,21.1,10.3
Top Pair,13.2,12.2,20.9,18.9,15.8,5.3,24.1
Overpair,1.1,0.0,1.2,2.5,0.6,5.3,0.0
Two Pair,11.3,17.9,15.1,15.9,24.7,31.6,20.7
Trips/Set,3.4,5.7,5.2,4.2,3.8,5.3,6.9
Straight,1.1,2.2,2.4,1.5,2.5,0.0,3.4


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,265.0,229.0,497.0,403.0,158.0,19.0,29.0
Air,41.5,41.5,32.2,30.8,31.6,10.5,10.3
Weak Pair,24.2,18.3,19.3,22.1,17.1,26.3,24.1
Top Pair,13.2,12.2,20.9,18.9,15.8,5.3,24.1
Overpair,1.1,0.0,1.2,2.5,0.6,5.3,0.0
Two Pair,11.3,17.9,15.1,15.9,24.7,31.6,20.7
Trips/Set,3.4,5.7,5.2,4.2,3.8,5.3,6.9
Monster,5.3,4.4,6.0,5.7,6.3,15.8,13.8
Draw,30.9,29.7,36.4,39.5,43.7,47.4,10.3


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,114.0,313.0,519.0,301.0,153.0,13.0,21.0
Air,62.3,49.5,48.0,50.5,35.3,61.5,42.9
Underpair,11.4,6.4,4.8,4.7,7.2,7.7,4.8
Bottom Pair,3.5,6.4,5.4,7.0,7.2,0.0,4.8
Middle Pair,3.5,7.3,11.0,8.3,11.1,0.0,14.3
Top Pair,7.0,7.7,8.7,10.6,9.2,15.4,9.5
Overpair,0.0,0.0,1.0,1.3,0.7,0.0,0.0
Two Pair,7.0,14.4,14.5,11.6,14.4,0.0,9.5
Trips/Set,5.3,3.8,3.7,3.0,6.5,7.7,9.5
Straight,0.0,0.6,1.7,0.7,3.9,7.7,0.0


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,114.0,313.0,519.0,301.0,153.0,13.0,21.0
Air,62.3,49.5,48.0,50.5,35.3,61.5,42.9
Weak Pair,18.4,20.1,21.2,19.9,25.5,7.7,23.8
Top Pair,7.0,7.7,8.7,10.6,9.2,15.4,9.5
Overpair,0.0,0.0,1.0,1.3,0.7,0.0,0.0
Two Pair,7.0,14.4,14.5,11.6,14.4,0.0,9.5
Trips/Set,5.3,3.8,3.7,3.0,6.5,7.7,9.5
Monster,0.0,4.5,3.1,3.0,8.5,7.7,4.8
Draw,30.7,31.3,32.0,34.9,32.0,53.8,42.9


In [17]:
from IPython.display import display

DRAW_FIELDS = [
    ('Flush Draw', 'responder_has_flush_draw'),
    ('OESD/DG', 'responder_has_oesd_dg'),
]

def _summarize_holdings(responses_subset):
    classified = responses_subset.dropna(subset=['responder_primary'])
    if classified.empty:
        return None
    total = len(classified)
    cat_counts = {cat: int((classified['responder_primary'] == cat).sum()) for cat in BASE_PRIMARY_CATEGORIES}

    base_index = ['Events'] + BASE_PRIMARY_CATEGORIES + [label for label, _ in DRAW_FIELDS]
    base_values = []
    for entry in base_index:
        if entry == 'Events':
            base_values.append(float(total))
        elif entry in BASE_PRIMARY_CATEGORIES:
            base_values.append(cat_counts.get(entry, 0) / total * 100 if total else 0.0)
        else:
            field = dict(DRAW_FIELDS)[entry]
            base_values.append(classified[field].mean() * 100 if total else 0.0)
    base_series = pd.Series(base_values, index=base_index, dtype=float)

    any_draw_pct = (classified['responder_has_flush_draw'] | classified['responder_has_oesd_dg']).mean() * 100 if total else 0.0
    group_index = ['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None]
    group_values = []
    for label in group_index:
        if label == 'Events':
            group_values.append(float(total))
        elif label == 'Draw':
            group_values.append(any_draw_pct)
        else:
            members = dict(GROUPED_PRIMARY_ORDER).get(label, [])
            group_values.append(sum(cat_counts.get(member, 0) for member in members) / total * 100 if total else 0.0)
    group_series = pd.Series(group_values, index=group_index, dtype=float)
    return base_series, group_series

if responses_df.empty:
    print('No c-bet response events available.')
else:
    raise_rows = responses_df[responses_df['response'] == 'Raise']
    call_rows = responses_df[responses_df['response'] == 'Call']

    raise_summary = _summarize_holdings(raise_rows) if not raise_rows.empty else None
    call_summary = _summarize_holdings(call_rows) if not call_rows.empty else None

    if not raise_summary and not call_summary:
        print('No raise or call responses with classified hands.')
    else:
        base_columns = []
        if raise_summary:
            base_columns.append(('C-Bet Raise Holdings (%)', raise_summary[0]))
        if call_summary:
            base_columns.append(('C-Bet Call Holdings (%)', call_summary[0]))

        if base_columns:
            base_df = pd.concat([series.rename(col) for col, series in base_columns], axis=1)
            base_df = base_df.reindex(['Events'] + BASE_PRIMARY_CATEGORIES + [label for label, _ in DRAW_FIELDS])
            styler = base_df.style
            value_rows = base_df.index.difference(['Events'])
            if not value_rows.empty:
                styler = styler.background_gradient(
                    cmap=_gradient_cmap('#d95f02'),
                    axis=None,
                    subset=pd.IndexSlice[value_rows, base_df.columns]
                )
            styler = (
                styler
                .format('{:.0f}', subset=pd.IndexSlice[['Events'], base_df.columns])
                .format('{:.1f}%', subset=pd.IndexSlice[value_rows, base_df.columns])
                .set_caption('C-Bet Raise vs Call Holdings')
            )
            display(styler)

        group_columns = []
        if raise_summary:
            group_columns.append(('C-Bet Raise Holdings (%)', raise_summary[1]))
        if call_summary:
            group_columns.append(('C-Bet Call Holdings (%)', call_summary[1]))

        if group_columns:
            group_df = pd.concat([series.rename(col) for col, series in group_columns], axis=1)
            group_df = group_df.reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
            group_styler = group_df.style
            group_value_rows = group_df.index.difference(['Events'])
            if not group_value_rows.empty:
                group_styler = group_styler.background_gradient(
                    cmap=_gradient_cmap('#3182bd'),
                    axis=None,
                    subset=pd.IndexSlice[group_value_rows, group_df.columns]
                )
            group_styler = (
                group_styler
                .format('{:.0f}', subset=pd.IndexSlice[['Events'], group_df.columns])
                .format('{:.1f}%', subset=pd.IndexSlice[group_value_rows, group_df.columns])
                .set_caption('C-Bet Raise vs Call Holdings (Grouped)')
            )
            display(group_styler)


,C-Bet Raise Holdings (%),C-Bet Call Holdings (%)
Events,669,3294
Air,33.6%,45.7%
Underpair,4.6%,7.8%
Bottom Pair,3.4%,6.8%
Middle Pair,5.4%,11.8%
Top Pair,23.6%,14.8%
Overpair,3.9%,1.2%
Two Pair,14.2%,6.6%
Trips/Set,7.9%,3.8%
Straight,1.6%,0.5%


,C-Bet Raise Holdings (%),C-Bet Call Holdings (%)
Events,669,3294
Air,33.6%,45.7%
Weak Pair,13.5%,26.4%
Top Pair,23.6%,14.8%
Overpair,3.9%,1.2%
Two Pair,14.2%,6.6%
Trips/Set,7.9%,3.8%
Monster,3.3%,1.4%
Draw,14.6%,14.2%
